In [1]:
library(sets)
library(tidyverse)
library(data.table)
library(hash)
`%ni%` = Negate(`%in%`)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ forcats::%>%()  masks stringr::%>%(), dplyr::%>%(), purrr::%>%(), tidyr::%>%(), tibble::%>%(), sets::%>%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


The following object is masked from ‘package:sets’:

    set


hash-2.2.6.2 provided by Decision Patterns



Attaching package: ‘hash’


The following object is masked from ‘package:data.table’:

    copy




In [2]:
# path to concept directories
path = ".../concepts/"
path = "/users/michael/desktop/chapter_10/data/concepts/"
concepts <- list.dirs(path=path, full = F, recursive = F)
# concepts <- concepts[! concepts %in% c("tmp", ".ipynb_checkpoints","TEMPLATES")]

# path to in/out-concept token file(s)
# annotated.tokens = read.csv(".../AnnotatedTokens.csv",row.names = 1)
annotated.tokens = read.csv("/users/michael/desktop/chapter_10/data/AnnotatedTokens.csv",row.names = 1)


In [3]:
variants <- list('STUDENT' = c('alumna','alumno','estudiante'),'FOOLISH ACT' = c('boludez','pavada'), 'HUSBAND' = c('esposo','marido'), 'NICKNAME' = c('apodo','sobrenombre','mote','remoquete'),'WAITER' = c('camarera', 'camarero','mesera','mesero'))

In [4]:
var2merge <- hash()
var2merge[['WAITER']] <- c('camarer*', 'meser*')
var2merge[['STUDENT']] <- 'alumn*'

# CALCULATE AGGREGATE

In [8]:
all.calcs <- data.frame()

for (concept in concepts){
    
    sizes <- list()
    u.values <- list() # uniformity values
    iu.values <- list() # internal uniformity
    
    csvFiles <- list.files(path=paste0(path,concept), pattern=".*csv$", full.names = TRUE)
    all.tokens <- annotated.tokens
    
    for (model in csvFiles){
        
        d <- read.csv(model, quote="",header=F)
        colnames(d) <- c("token","cluster")
        
        model.name <- strsplit(model, "/")[[1]][9]
        model <- str_replace_all(model.name, ".tcmx.soc.pac__cluster_assignment.csv","")

        noise.ratio <- (nrow(d[which(d$cluster == -1),])/nrow(d))*100
        number.clusters <- nlevels(as.factor(d$cluster)) #- 1

        d.token <- data.frame(do.call("rbind", strsplit(as.character(d$token),"/", fixed=TRUE)))
        d.new <- cbind.data.frame(d, d.token)
        colnames(d.new)[3:6] <- c("type", "pos", "lect", "id")

        d.lect <- data.frame(do.call("rbind", strsplit(as.character(d.new$lect), "_|[0-9]"))[,1:2]) 
        colnames(d.lect) <- c("country", "register")    
        d.final <- cbind.data.frame(d.new, d.lect) 
        d.final <- subset(d.final, cluster != -1)

        cat("----------------------\n",
            concept, "\n", 
            "Model:", model, "\n",
            "Noise:", noise.ratio, "\n",
            "Total clusters:", number.clusters, "\n",
           "----------------------\n")

        tab <- table(d.final$cluster, d.final$country)

        # remove monolectal==========================================================================
        monolectal.clusters <- rownames(tab)[rowSums(tab == 0) == nlevels(d.final$country) - 1]
        d.final.sub <- subset(d.final, d.final$cluster %ni% monolectal.clusters)
        cat("\nMonolectal cluster(s):", monolectal.clusters, "\n\n")

        # only clusters with >5 annotated tokens =====================================================
        ann.clusters <- subset(d.final.sub$cluster, d.final.sub$token %in% rownames(all.tokens))
        values <- c()
        for (value in ann.clusters){
            if (sum(ann.clusters == value) >= 5){
                values<-c(values, as.numeric(value))
            }
        }
        values <- as.set(values)
        only_ann <- subset(d.final.sub, d.final.sub$cluster %in% values)
        ann.toks <- subset(only_ann, only_ann$token %in% rownames(all.tokens))

        # keeping clusters with >80% in concept annotated tokens =======================================
        
        clust2include <- c()
        for (clust in values){
            (cat("===========================\n"))
            (cat("Cluster:", clust,"\n"))
            toks.cluster <- subset(ann.toks, as.numeric(ann.toks$cluster) == clust)

            if(length(unique(toks.cluster$type)) == 1){
                cat("Cluster", clust, "is monolexical.", "\n")
            }

            ann.toks.sub <- subset(all.tokens, rownames(all.tokens) %in% toks.cluster$token)
            ins <- sum(ann.toks.sub$in_concept == 1)
            total <- length(ann.toks.sub$in_concept)
            ins_perc <- round(as.numeric(ins/total)*100)
            (cat(ins_perc, "%  (",ins, ") in concept\n"))
            (cat("There are", nrow(ann.toks.sub),"total annotated tokens in cluster",clust, "\n"))

            if (ins_perc > 80){
                (cat("KEEP CLUSTER",clust, "\n"))
                clust2include <- c(clust2include,clust) 

            }else{
                (cat("**DISCARD** cluster", clust, "\n"))
            }
        (cat("===========================\n\n"))

        }

        flush.console()
        
        cat(length(clust2include), "cluster(s) kept for", model,"\n\n\n") 

        if (length(clust2include) == 0){
            cat("Go to next model. No clusters in", model,"\n")
            next
        }


        flush.console()

        # subsetting for clusters not discarded =========================================================

        d.final.in <- subset(d.final.sub, d.final.sub$cluster %in% clust2include)

        # calculate uniformity values ================================================================

        countries <- c("arg","col", "mex", "per", "spa", "usa")
        lectal.comp <- c()
        
        for (lect in countries){
            for (lect2 in countries){
                if (lect != lect2){

                    if (paste0(lect2,"-",lect) %in% lectal.comp){
                        next
                    }

                    country.comparison <- c(lect,lect2)
                    dsub <- droplevels(subset(d.final.in, country %in% country.comparison))

                    if (concept %in% keys(var2merge)){
                        dsub2 <- mutate(dsub, type2 = factor(str_replace(type, "(.+)[aoe]$", "\\1"))) 
                        utab <- table(dsub2$type2, dsub2$country)
                    }else{
                        utab <- table(dsub$type, dsub$country)
                    }
                    
                    if (nrow(utab) == 1){
                        monolex = TRUE
                    }else{
                        monolex = FALSE
                    }

                    if (ncol(utab) == 0){
                        u <- NA
                        lect.iu <- NA
                        lect2.iu <- NA
                    }else if(ncol(utab) == 1){
                        u <- NA
                        if (colnames(utab) == lect){
                            lect.iu <- round(sum(prop.table(utab[,1])^2), 10)
                            lect2.iu <- NA
                        }else{
                            lect2.iu <- round(sum(prop.table(utab[,1])^2), 10)
                            lect.iu <- NA
                        }
                    }else if (monolex == FALSE){
                        if (fisher.test(utab, simulate.p.value = TRUE)$p > 0.05){  
                            u <- 1
                            lect.iu <- round(sum(prop.table(utab[,1])^2), 10)
                            lect2.iu <- round(sum(prop.table(utab[,2])^2), 10)
                        }else{
                            prop.utab <- prop.table(utab, 2)
                            u <- round(sum(apply(prop.utab, 1, min), na.rm=T), 10)
                            lect.iu <- round(sum(prop.table(utab[,1])^2), 10)
                            lect2.iu <- round(sum(prop.table(utab[,2])^2), 10)  
                        } 
                    }else{
                        prop.utab <- prop.table(utab, 2)
                        u <- round(sum(apply(prop.utab, 1, min), na.rm=T), 10)
                        lect.iu <- round(sum(prop.table(utab[,1])^2), 10)
                        lect2.iu <- round(sum(prop.table(utab[,2])^2), 10)  
                    }
                        
                    sizes[[model]][paste0(lect,"-",lect2)] <- sum(utab)
                    u.values[[model]][paste0(lect,"-",lect2)] <- u  
                    iu.values[[model]][lect] <- lect.iu
                    iu.values[[model]][lect2] <- lect2.iu   
                    lectal.comp <- c(lectal.comp,paste0(lect,"-",lect2))

                }
            }
        }
        
    }
    
    if (length(u.values)==0){
        next
    }
    
    
    weight.u.means <- c()
    u.means <- c()
    size.means <- c()
    sd.u.values <- c()
    
    for (lects in lectal.comp){
        values <- c()
        for (i in 1:length(u.values)){
            if (lects %in% names(u.values[[i]])){
                values <- c(values,u.values[[i]][[lects]])
            }else{ 
                next
            }
        }
        avg.values <- mean(values,na.rm = TRUE)
        sd.u.values[paste0(lects," SD(u.values)")] <- sd(values,na.rm = TRUE)
        u.means[paste0(lects," (u.means)")] <- avg.values
        
        size.values <- c()
        for (i in 1:length(sizes)){
            if (lects %in% names(sizes[[i]])){
                size.values <- c(size.values,sizes[[i]][[lects]])
            }else{
                next

            }
        }
        avg.size <- mean(size.values,na.rm = TRUE)
        size.means[paste0(lects," (size.means)")] <- avg.size
        wm <- weighted.mean(values,size.values,na.rm = TRUE)
        weight.u.means[paste0(lects," (weight.u.means)")] <- wm
    }

    # calculate mean internal uniformity
    iu.means <- list()
    for (c in countries){
        vals <- c()
        for (i in 1:length(iu.values)){
            if (c %in% names(iu.values[[i]])){
                vals <- c(vals,iu.values[[i]][[c]])                
            }
        }
        iu.means[paste0(c, " (iu.means)")] <- mean(vals, na.rm = TRUE)
    }
    
    # merge everything into one dataframe
    df1 <- data.frame(colnames=names(u.means))
    df1 <- rbind(u.means)
    df2 <- data.frame(colnames=names(weight.u.means))
    df2 <- rbind(weight.u.means)
    df2.2 <- data.frame(colnames=names(iu.means))
    df2.2 <- rbind(iu.means)
    df3 <- data.frame(colnames=names(size.means))
    df3 <- rbind(size.means)
    df4 <- data.frame(colnames = names(sd.u.values))
    df4 <- rbind(sd.u.values)
    df.all <- cbind(df1,df4,df2.2,df2,df3)
    rownames(df.all) <- NULL
    `models (total)` <- length(u.values)
    
    all.calcs <- rbind(all.calcs, cbind.data.frame(concept,`models (total)`,df.all))

}
all.calcs

----------------------
 FOOLISH ACT 
 Model: FOOLISH ACT.nobound15-15openclass.PPMIweight.LENGTH5000.SOCPOSall 
 Noise: 0 
 Total clusters: 2 
 ----------------------

Monolectal cluster(s):  

Cluster: 1 
90 %  ( 161 ) in concept
There are 178 total annotated tokens in cluster 1 
KEEP CLUSTER 1 

1 cluster(s) kept for FOOLISH ACT.nobound15-15openclass.PPMIweight.LENGTH5000.SOCPOSall 


----------------------
 FOOLISH ACT 
 Model: FOOLISH ACT.nobound5-5all.PPMIno.LENGTH5000.SOCPOSall 
 Noise: 2.593193 
 Total clusters: 3 
 ----------------------

Monolectal cluster(s):  

Cluster: 0 
92 %  ( 47 ) in concept
There are 51 total annotated tokens in cluster 0 
KEEP CLUSTER 0 

Cluster: 1 
90 %  ( 111 ) in concept
There are 124 total annotated tokens in cluster 1 
KEEP CLUSTER 1 

2 cluster(s) kept for FOOLISH ACT.nobound5-5all.PPMIno.LENGTH5000.SOCPOSall 


----------------------
 HUSBAND 
 Model: HUSBAND.nobound10-10all.PPMIselection.LENGTH5000.SOCPOSall 
 Noise: 0 
 Total clusters: 4 
 -

concept,models (total),arg-col (u.means),arg-mex (u.means),arg-per (u.means),arg-spa (u.means),arg-usa (u.means),col-mex (u.means),col-per (u.means),col-spa (u.means),⋯,col-mex (size.means),col-per (size.means),col-spa (size.means),col-usa (size.means),mex-per (size.means),mex-spa (size.means),mex-usa (size.means),per-spa (size.means),per-usa (size.means),spa-usa (size.means)
<chr>,<int>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,⋯,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
FOOLISH ACT,2,1,1,0.7599046,1,1,1,1,1,⋯,32,42,31,61.5,52,41,71.5,51,81.5,70.5
HUSBAND,2,0.6944297,0.7345449,0.6417998,0.9447213,0.8118299,0.9598847,0.9473701,0.639151,⋯,3926.5,3409.5,3391.5,3684.5,4288,4270,4563,3753,4046,4028
NICKNAME,2,0.955251,1,1,1,1,0.9166519,0.9050033,0.9010723,⋯,457.5,483,398,393,523.5,438.5,433.5,464,459,374
STUDENT,2,0.6924982,1,0.9849733,0.9098427,0.856962,0.6907469,0.6855648,0.602341,⋯,2977,3245,2703,2639.5,2883,2341,2277.5,2609,2545.5,2003.5
WAITER,2,0.4571747,0.5180261,0.587924,0.8739194,0.851359,1,0.9327122,0.3310941,⋯,365,247.5,421,373.5,264.5,438,390.5,320.5,273,446.5


In [ ]:
write.csv(as.matrix(all.calcs),".../modeled_Uvalues.csv", row.names = FALSE)